### STEP 1 RAW 데이터 불러오기
원본 train 데이터인 train.csv와 우리 팀이 생성한 일반데이터인 'general.csv'를 불러와 'train_general_combined.csv'로 병합합니다.

'train_general_combined.csv' 데이터는 /output/ 내에 저장합니다.

In [1]:
import pandas as pd

# CSV 파일 불러오기
train = pd.read_csv("./filestore_raw/train.csv")     # 첫 번째 CSV
general = pd.read_csv("./filestore_raw/general.csv") # 두 번째 CSV

# 두 DataFrame 합치기 (train 다음에 general)
combined = pd.concat([train, general], ignore_index=True)

output_path = "./output/train_general_combined.csv"
combined.to_csv(output_path, index=False)

print(f"✅ 병합 완료! 저장 경로: {output_path}")
print(f"총 행 개수: {len(combined)}")

✅ 병합 완료! 저장 경로: ./output/train_general_combined.csv
총 행 개수: 4750


### STEP2 데이터를 로드하기
csv에서 판다스로 데이터를 로드하고 기본 통계와 줄바꿈 패턴을 저장합니다.

In [2]:
import numpy as np
import re
from typing import List
import random

# 랜덤 시드 고정 (재현성)
random.seed(42)
np.random.seed(42)

df = pd.read_csv('./output/train_general_combined.csv')

print(f"\n전체 데이터 수: {len(df):,}개")
print(f"컬럼: {df.columns.tolist()}")
print(f"\n클래스 분포:")
print(df['class'].value_counts())

# 줄바꿈 패턴 분석
df['newline_count'] = df['conversation'].str.count('\n')
print(f"\n줄바꿈 통계:")
print(f"\n- 줄바꿈 없는 데이터: {(df['newline_count'] == 0).sum():,}개")
print(f"- 줄바꿈 있는 데이터: {(df['newline_count'] > 0).sum():,}개")

print(f"✅ 로드 완료")


전체 데이터 수: 4,750개
컬럼: ['idx', 'class', 'conversation']

클래스 분포:
class
기타 괴롭힘 대화      1094
갈취 대화           981
직장 내 괴롭힘 대화     979
협박 대화           896
일반              800
Name: count, dtype: int64

줄바꿈 통계:

- 줄바꿈 없는 데이터: 800개
- 줄바꿈 있는 데이터: 3,950개
✅ 로드 완료


### STEP3. 전처리 함수를 정의합니다.
*여러 방법을 실험하여 최적의 방법을 찾을 수 있도록 세가지 방법으로 전처리하였습니다.*

(1) 줄바꿈을 완전 제거하여 한 줄로 만들기 -> 줄바꿈을 공백으로 대체합니다.
- 모든 줄바꿈(\n)을 공백으로 대체하고 공백이 복수 개인 경우 단일 공백으로 대체합니다.
- 이 경우에는 추후 TEST데이터 분석시에도 동일하게 줄바꿈을 공백으로 대체하여 test해야 할 것으로 보입니다.

(2) 줄바꿈을 구분 토큰인 [SEP]로 대체
- 줄바꿈(\n)을 특수 구분 토큰(기본값: [SEP])으로 대체
- 대화 턴의 구조 정보 보존
- 모델이 턴 구분을 학습할 수 있도록 함
    
(3) GPT로 생성된 일반 대화 데이터에 임의 줄바꿈 추가
- 단일 문장 데이터를 문장 부호(., ?, !) 기준으로  여러 줄로 분할
- 데이터 균형 확보 (train과 test 간 포맷 차이 완화)


In [3]:
""" [전략1] """
def preprocess_strategy_1(text: str) -> str:

    # 줄바꿈을 공백으로 대체
    processed = text.replace('\n', ' ')
    
    # 다중 공백을 단일 공백으로 정리
    processed = re.sub(r'\s+', ' ', processed)
    
    # 앞뒤 공백 제거
    processed = processed.strip()
    
    return processed

""" [전략2] """
import re

def preprocess_strategy_2(text: str, separator: str = '[SEP]') -> str:
    """
    - 텍스트에 줄바꿈(\n)이 하나라도 있으면: 줄바꿈만 구분자로 사용.
    - 줄바꿈이 전혀 없으면: 문장 끝 문장부호(.!?… 등) 뒤에 구분자 삽입.
    - 결과는 중복된 구분자와 과도한 공백을 정리한 한 줄 문자열로 반환.
    """
    if not text:
        return ''

    # Normalize newlines
    t = text.replace('\r\n', '\n').replace('\r', '\n')

    if '\n' in t:
        # 줄바꿈만 구분자로 처리 (연속 줄바꿈은 하나로)
        processed = re.sub(r'\n+', f' {separator} ', t)
    else:
        # 줄바꿈 없음 -> 문장부호 기준으로 분리
        # 문장부호 뒤에 구분자 추가 (따옴표/괄호 닫힘 포함)
        processed = re.sub(
            r'([\.!?。！？…]+)([)"\'”’\]]*)\s*',
            rf'\1\2 {separator} ',
            t
        )

    # 연속된 구분자/공백을 하나로 축약
    processed = re.sub(rf'(?:\s*{re.escape(separator)}\s*)+', f' {separator} ', processed)
    # 다중 공백 정리 및 앞뒤 공백 제거
    processed = re.sub(r'\s+', ' ', processed).strip()

    return processed


""" [전략3] """
def add_random_newlines(text: str, min_lines: int = 3, max_lines: int = 10) -> str:

    # 이미 줄바꿈이 충분히 있으면 그대로 반환
    if text.count('\n') >= min_lines:
        return text
    
    # 문장 부호 기준으로 분할 가능한 위치 찾기
    sentences = re.split(r'([.?!]\s+)', text)
    
    # 분할 결과가 너무 적으면 원본 반환
    if len(sentences) < 3:
        return text
    
    # 문장과 구분자 재조합
    reconstructed = []
    for i in range(0, len(sentences)-1, 2):
        if i+1 < len(sentences):
            reconstructed.append(sentences[i] + sentences[i+1].strip())
        else:
            reconstructed.append(sentences[i])
    
    # 마지막 요소 처리
    if len(sentences) % 2 == 1:
        reconstructed.append(sentences[-1])
    
    # 줄바꿈 추가할 개수 결정
    num_newlines = min(random.randint(min_lines, max_lines), len(reconstructed)-1)
    
    # 랜덤하게 줄바꿈 삽입
    if len(reconstructed) > 1:
        # 줄바꿈 위치 랜덤 선택
        split_indices = sorted(random.sample(range(1, len(reconstructed)), 
                                           min(num_newlines, len(reconstructed)-1)))
        
        result_parts = []
        prev_idx = 0
        for idx in split_indices:
            result_parts.append(' '.join(reconstructed[prev_idx:idx]))
            prev_idx = idx
        result_parts.append(' '.join(reconstructed[prev_idx:]))
        
        return '\n'.join(result_parts)
    
    return text

print(f"✅ 전처리 함수 정의 완료")

✅ 전처리 함수 정의 완료


### STEP 4. 전처리를 적용합니다.

In [4]:
# 전략 1: 줄바꿈 완전 제거
df_strategy1 = df.copy()
df_strategy1['conversation_processed'] = df_strategy1['conversation'].apply(preprocess_strategy_1)
print(f"✅ 전략 1 완료")

# 전략 2: 구분 토큰으로 대체
df_strategy2 = df.copy()
df_strategy2['conversation_processed'] = df_strategy2['conversation'].apply(
    lambda x: preprocess_strategy_2(x, separator='[SEP]')
)
print(f"✅ 전략 2 완료")


# 전략 3: 일반 대화에 줄바꿈 추가
df_strategy3 = df.copy()

# 일반 대화(줄바꿈 없는 데이터)에만 줄바꿈 추가
mask_general = df_strategy3['newline_count'] == 0
df_strategy3.loc[mask_general, 'conversation_processed'] = df_strategy3.loc[mask_general, 'conversation'].apply(
    add_random_newlines
)
df_strategy3.loc[~mask_general, 'conversation_processed'] = df_strategy3.loc[~mask_general, 'conversation']
print(f"✅ 전략3 완료")

print("\n✅ 전처리 완료!")

✅ 전략 1 완료
✅ 전략 2 완료
✅ 전략3 완료

✅ 전처리 완료!


### STEP4-1. 클래스별 줄바꿈 비율 맞추기
전략 3 결과가 만들어진 뒤, 클래스마다 절반은 줄바꿈 상태를 유지/추가하고 나머지는 줄바꿈을 제거해 포맷 균형을 맞춥니다.


In [5]:
# 전략 3 결과에서 클래스별 줄바꿈/무줄바꿈을 균형 있게 분배
df_strategy3['conversation_processed'] = df_strategy3['conversation_processed'].fillna('')
rng = np.random.RandomState(42)
for cls_value, idx in df_strategy3.groupby('class').groups.items():
    cls_indices = np.array(list(idx))
    if len(cls_indices) == 0:
        continue
    rng.shuffle(cls_indices)
    newline_count = int(np.ceil(len(cls_indices) / 2))
    newline_idx = cls_indices[:newline_count]
    flat_idx = cls_indices[newline_count:]
    df_strategy3.loc[newline_idx, 'conversation_processed'] = (
        df_strategy3.loc[newline_idx, 'conversation_processed']
        .apply(add_random_newlines)
    )
    df_strategy3.loc[flat_idx, 'conversation_processed'] = (
        df_strategy3.loc[flat_idx, 'conversation_processed']
        .apply(preprocess_strategy_1)
    )

newline_stats = (
    df_strategy3.assign(
        has_newline=df_strategy3['conversation_processed'].str.contains('\n', regex=False)
    )
    .groupby('class')['has_newline']
    .agg(['sum', 'count'])
)
newline_stats.rename(columns={'sum': 'newline_rows', 'count': 'total_rows'}, inplace=True)
newline_stats['no_newline_rows'] = newline_stats['total_rows'] - newline_stats['newline_rows']
print(newline_stats)
print('✅ 클래스별 줄바꿈/무줄바꿈 분배 완료')


             newline_rows  total_rows  no_newline_rows
class                                                 
갈취 대화                 491         981              490
기타 괴롭힘 대화             547        1094              547
일반                    400         800              400
직장 내 괴롭힘 대화           490         979              489
협박 대화                 448         896              448
✅ 클래스별 줄바꿈/무줄바꿈 분배 완료


In [6]:
### STEP 5 전처리 데이터를 csv로 출력합니다.


In [7]:

# 각 전략별 저장
output_columns = ['idx', 'class', 'conversation_processed']

df_strategy1_output = df_strategy1[output_columns].rename(
    columns={'conversation_processed': 'conversation'}
)
df_strategy1_output.to_csv('./output/train_strategy1_remove_newlines.csv', index=False, encoding='utf-8-sig')
print("\n✓ 전략 1 저장 완료: ./output/train_strategy1_remove_newlines.csv")

df_strategy2_output = df_strategy2[output_columns].rename(
    columns={'conversation_processed': 'conversation'}
)
df_strategy2_output.to_csv('./output/train_strategy2_separator_token.csv', index=False, encoding='utf-8-sig')
print("✅  전략 2 저장 완료: ./output/train_strategy2_separator_token.csv")

df_strategy3_output = df_strategy3[output_columns].rename(
    columns={'conversation_processed': 'conversation'}
)
df_strategy3_output.to_csv('./output/train_strategy3_add_random_newlines.csv', index=False, encoding='utf-8-sig')
print("✅  전략 3 저장 완료: ./output/train_strategy3_add_random_newlines.csv")


✓ 전략 1 저장 완료: ./output/train_strategy1_remove_newlines.csv
✅  전략 2 저장 완료: ./output/train_strategy2_separator_token.csv
✅  전략 3 저장 완료: ./output/train_strategy3_add_random_newlines.csv
